### Import packages correction

In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import tensorflow as tf

# Get your datasets

In [2]:
from data.dataset import H5PY_RW
from data.data_tools import read_data_header, males_keys, females_keys

H5_dic = read_data_header()
chunk_size = 512*30

males = H5PY_RW().open_h5_dataset('test_raw.h5py', subset = males_keys(H5_dic)).set_chunk(chunk_size).shuffle()
fem = H5PY_RW().open_h5_dataset('test_raw.h5py', subset = females_keys(H5_dic)).set_chunk(chunk_size).shuffle()
print 'Data with', len(H5_dic), 'male and female speakers'


Data with 40 male and female speakers


# Mixing the dataset

In [3]:
from data.dataset import Mixer

mixed_data = Mixer([males, fem], with_mask=False, with_inputs=True)

# Training set selection
mixed_data.select_split(0)

# Model pretrained loading

In [4]:
learning_rate = 0.0001
batch_size = 1
N = 256
max_pool = 256
regularization = 0.0001

config_model = {}
config_model["alpha"] = learning_rate
config_model["reg"] = regularization
config_model["batch_size"] = batch_size
config_model["chunk_size"] = chunk_size
config_model["N"] = N
config_model["maxpool"] = max_pool
config_model["type"] = "pretraining"
config_model["smooth_size"] = 10
idd = ''.join('-{}={}-'.format(key, val) for key, val in sorted(config_model.items()))
config_model["type"] = "DPCL_train_front"

In [5]:
from models.adapt import Adapt

full_id = 'tight-base-9844'+idd

# Create a new Adapt model with the same weights as the one loaded
model = Adapt.load(config_model, full_id, pretraining=False, folder='pretraining', new_folder='DPCL_with_front')

ID : AdaptiveNet-white-mouse-4413
INFO:tensorflow:Restoring parameters from /home/anthony/das/log/pretraining/AdaptiveNet-tight-base-9844-N=256--alpha=0.0001--batch_size=1--chunk_size=15360--maxpool=256--reg=0.0001--smooth_size=10--type=pretraining-/model.ckpt


## Connect DPCL model to the front end

In [6]:
from models.dpcl import DPCL

with model.graph.as_default():
    model.connect_front(DPCL)
    model.freeze_front()
    model.optimize
    model.tensorboard_init()

Tensor("prediction/l2_normalize:0", shape=(?, ?, 256, 40), dtype=float32)
[<tf.Variable 'prediction/W:0' shape=(1, 600, 10240) dtype=float32_ref>, <tf.Variable 'prediction/b:0' shape=(10240,) dtype=float32_ref>, <tf.Variable 'prediction/forward_BLSTM_1/rnn/basic_lstm_cell/kernel:0' shape=(556, 1200) dtype=float32_ref>, <tf.Variable 'prediction/forward_BLSTM_1/rnn/basic_lstm_cell/bias:0' shape=(1200,) dtype=float32_ref>, <tf.Variable 'prediction/backward_BLSTM_1/rnn/basic_lstm_cell/kernel:0' shape=(556, 1200) dtype=float32_ref>, <tf.Variable 'prediction/backward_BLSTM_1/rnn/basic_lstm_cell/bias:0' shape=(1200,) dtype=float32_ref>, <tf.Variable 'prediction/forward_BLSTM_2/rnn/basic_lstm_cell/kernel:0' shape=(900, 1200) dtype=float32_ref>, <tf.Variable 'prediction/forward_BLSTM_2/rnn/basic_lstm_cell/bias:0' shape=(1200,) dtype=float32_ref>, <tf.Variable 'prediction/backward_BLSTM_2/rnn/basic_lstm_cell/kernel:0' shape=(900, 1200) dtype=float32_ref>, <tf.Variable 'prediction/backward_BLSTM_

In [ ]:
with model.graph.as_default():
    uninit =[v for v in tf.global_variables() if v.name.split(':')[0] in set(model.sess.run(tf.report_uninitialized_variables()))]
    init = tf.variables_initializer(uninit)

# Model creation

# Pretraining the model 

In [ ]:
nb_iterations = 1000

#initialize the model
model.sess.run(init)

for i in range(nb_iterations):
    X_in, X_mix, Ind = mixed_data.get_batch(batch_size)
    c = model.train(X_mix, X_in, learning_rate, i)
    print 'Step #'  ,i,' loss=', c 

    if i%20 == 0: #cost_valid < cost_valid_min:
        print 'DAS model saved at iteration number ', i,' with cost = ', c 
        model.save(i)

Step # 0  loss= 136.977
DAS model saved at iteration number  0  with cost =  136.977
Step # 1  loss= 127.23
Step # 2  loss= 120.381
Step # 3  loss= 115.498
Step # 4  loss= 111.526
Step # 5  loss= 107.752
Step # 6  loss= 105.123
Step # 7  loss= 102.107
Step # 8  loss= 101.1
Step # 9  loss= 99.2566
Step # 10  loss= 97.9084
Step # 11  loss= 96.6811
Step # 12  loss= 95.8864
Step # 13  loss= 94.7904
Step # 14  loss= 94.4061
Step # 15  loss= 93.2715
Step # 16  loss= 93.3088
Step # 17  loss= 91.9099
Step # 18  loss= 92.3574
Step # 19  loss= 91.9027
Step # 20  loss= 90.4495
DAS model saved at iteration number  20  with cost =  90.4495
Step # 21  loss= 91.009
Step # 22  loss= 88.4637
Step # 23  loss= 90.3575
Step # 24  loss= 90.1031
Step # 25  loss= 89.7105
Step # 26  loss= 89.2782
Step # 27  loss= 89.2154
Step # 28  loss= 88.9979
Step # 29  loss= 88.5978
Step # 30  loss= 88.5556
Step # 31  loss= 87.9523
Step # 32  loss= 88.1287
Step # 33  loss= 87.9291
Step # 34  loss= 87.7071
Step # 35  loss=

Step # 285  loss= 71.8593
Step # 286  loss= 71.4647
Step # 287  loss= 71.921
Step # 288  loss= 71.8797
Step # 289  loss= 71.824
Step # 290  loss= 71.2517
Step # 291  loss= 71.7496
Step # 292  loss= 71.5946
Step # 293  loss= 71.3426
Step # 294  loss= 71.699
Step # 295  loss= 70.266
Step # 296  loss= 70.6445
Step # 297  loss= 71.291
Step # 298  loss= 71.4149
Step # 299  loss= 71.3478
Step # 300  loss= 71.2651
DAS model saved at iteration number  300  with cost =  71.2651
Step # 301  loss= 71.1831
Step # 302  loss= 70.0403
Step # 303  loss= 71.2401
Step # 304  loss= 71.2268
Step # 305  loss= 71.1784
Step # 306  loss= 71.0043
Step # 307  loss= 70.9126
Step # 308  loss= 70.5564
Step # 309  loss= 70.9298
Step # 310  loss= 70.9566
Step # 311  loss= 70.9331
Step # 312  loss= 70.8222
Step # 313  loss= 70.7823
Step # 314  loss= 70.2963
Step # 315  loss= 70.7368
Step # 316  loss= 70.4141
Step # 317  loss= 70.6324
Step # 318  loss= 70.5887
Step # 319  loss= 70.2459
Step # 320  loss= 70.4399
DAS mo

Step # 568  loss= 60.8308
Step # 569  loss= 60.8504
Step # 570  loss= 60.2308
Step # 571  loss= 60.5515
Step # 572  loss= 60.5354
Step # 573  loss= 56.9195
Step # 574  loss= 60.6896
Step # 575  loss= 60.5611
Step # 576  loss= 59.7433
Step # 577  loss= 60.4441
Step # 578  loss= 60.5616
Step # 579  loss= 60.454
Step # 580  loss= 60.4074
DAS model saved at iteration number  580  with cost =  60.4074
Step # 581  loss= 60.3504
Step # 582  loss= 60.4041
Step # 583  loss= 59.9669
Step # 584  loss= 59.8842
Step # 585  loss= 60.3274
Step # 586  loss= 58.0953
Step # 587  loss= 59.602
Step # 588  loss= 60.0765
Step # 589  loss= 60.1848
Step # 590  loss= 60.1357
Step # 591  loss= 59.6424
Step # 592  loss= 60.0857
Step # 593  loss= 58.5547
Step # 594  loss= 59.9973
Step # 595  loss= 59.7727
Step # 596  loss= 59.9514
Step # 597  loss= 59.9295
Step # 598  loss= 59.1637
Step # 599  loss= 59.5922
Step # 600  loss= 59.8222
DAS model saved at iteration number  600  with cost =  59.8222
Step # 601  loss= 